In [1]:
import sys
import os
import pathlib
sys.path.insert(0, str(pathlib.Path(
).resolve().parent)) #.. this has to be the folder reCNN_visual_prosthesis
import wandb
import glob
from models import reCNN_bottleneck_CyclicGauss3d_no_scaling, Lurz_Control_Model
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
from Antolik_dataset import AntolikDataset, AntolikDataModule
from experiments.utils import pickle_read, download_model, get_neuron_estimates, reconstruct_orientation_maps, visualize_preferred_orientations, rotate_and_compute_distances, circular_distance
import pytorch_lightning as pl
import torch
from model_trainer import run_wandb_training
from energy_model.energy_model import EnergyModel, EnergyModelIndividual
from model_trainer import Antolik_dataset_preparation_function, get_model
from utils import get_config

import torch.nn as nn
import torch.optim as optim
from experiments.utils import pickle_read, download_model
from tqdm import tqdm
# %load_ext autoreload
# %autoreload 2
from imagen.image import BoundingBox
from experiments.MEI import *
import imagen
import pandas as pd
import plotly.graph_objects as go

### TIP: The `imagen` library produces a lot of warnings. If the number of warnings is too big (when you generate the data, for example), click on the output of the cell and press `o` to suppress the output of the cell.

In [2]:
bottleneck_model = download_model(artifact_name="reCNN_bottleneck_CyclicGauss3d:v1011", model_class=reCNN_bottleneck_CyclicGauss3d_no_scaling)
energy_model = download_model(artifact_name="EnergyModel:v556", model_class=EnergyModel)
encoder = download_model(artifact_name="StackedCore_FullGaussian2d:v2009", model_class=Lurz_Control_Model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: csng-cuni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Global seed set to 42
/opt/conda/lib/python3.8/site-packages/neuralpredictors/measures/modules.py:78: UserWarning: Poissonloss is averaged per batch. It's recommended to use `sum` instead
  warnings.warn("Poissonloss is averaged per batch. It's recommended to use `sum` instead")
/opt/conda/lib/python3.8/site-packages/neuralpredictors/layers/cores/conv2d.py:128: UserWarning: The averaged value of regularizer will be used.
  warnings.warn("The averaged value of regularizer will be used.", UserWarning)


Model from ./artifacts/reCNN_bottleneck_CyclicGauss3d:v1011/epoch=13-step=55999.ckpt loaded!


/auto/budejovice1/mpicek/reCNN_visual_prosthesis/readout.py:344: UserWarning: sigma is sampled from uniform distribution, instead of a fixed value. Consider setting fixed_sigma to True
  warnings.warn(


wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

Model from ./artifacts/EnergyModel:v556/epoch=15-step=63999.ckpt loaded!


wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Global seed set to 42


Model from ./artifacts/StackedCore_FullGaussian2d:v2009/epoch=4-step=1999.ckpt loaded!


/opt/conda/lib/python3.8/site-packages/neuralpredictors/measures/modules.py:78: UserWarning: Poissonloss is averaged per batch. It's recommended to use `sum` instead
  warnings.warn("Poissonloss is averaged per batch. It's recommended to use `sum` instead")
/opt/conda/lib/python3.8/site-packages/neuralpredictors/layers/cores/conv2d.py:128: UserWarning: The averaged value of regularizer will be used.
  warnings.warn("The averaged value of regularizer will be used.", UserWarning)


In [3]:
config = get_config()
config["train_data_dir"] = "/storage/brno2/home/mpicek/reCNN_visual_prosthesis/data/antolik_reparametrized_small/one_trials.pickle"
config["test_data_dir"] = "/storage/brno2/home/mpicek/reCNN_visual_prosthesis/data/antolik_reparametrized_small/ten_trials.pickle"

dm = Antolik_dataset_preparation_function(config, load_data=True)

pos_x, pos_y, orientations = dm.get_ground_truth(
    ground_truth_positions_file_path=config["ground_truth_positions_file_path"], 
    ground_truth_orientations_file_path=config["ground_truth_orientations_file_path"], 
    in_degrees=False, 
    positions_minus_y=config["positions_minus_y"], 
    positions_minus_x=config["positions_minus_x"], 
    positions_swap_axes=config["positions_swap_axes"]
)

Data loaded successfully!
Loaded precomputed mean from /storage/brno2/home/mpicek/reCNN_visual_prosthesis/data/antolik_reparametrized_small/one_trials_mean.npy


In [4]:
def generate_data_first_n_neurons(model, phase, frequency, neurons=10):
    """
    Generates gratings and returns the responses in shape (neurons, neurons)
    Beware, responses of all neurons are returned, but only gratings for the first neurons are
    presented (and are stacked into one batch of size neurons, thus the shape (neurons, neurons))
    """
    
    x_size_in_degrees = 11
    x_size = 55
    y_size = 55
    y_size_in_degrees = 11
    inputs = []
    for i in range(neurons):
        inp = imagen.SineGrating(
            orientation = orientations[i],
            frequency = frequency,
            phase = phase,
            bounds = BoundingBox(radius=x_size_in_degrees/2) ,
            offset = 0,
            scale = 1,
            xdensity = x_size/x_size_in_degrees,
            ydensity = y_size/y_size_in_degrees,
        )()
        scaled_array = (inp * (2 + 1.7)) -1.7
        scaled_array = torch.Tensor(scaled_array)
        inputs.append(scaled_array)

    inputs = torch.stack(inputs)
    responses = model(inputs.reshape(neurons,1,55,55).to(energy_model.device))
    responses = responses[:, :neurons]
    return responses.cpu().detach().numpy()

In [5]:
neurons = 10
num_phases = 10
num_freqs = 30

dataframes = []

for phase in np.linspace(0, 2*np.pi, num_phases):
    for freq in np.linspace(0.3, 1.8, num_freqs):
        for model in ["BM", "EM", "encoder"]:
            data = None
            if model == "BM":
                data = generate_data_first_n_neurons(bottleneck_model, phase, freq, neurons)
            elif model == "EM":
                data = generate_data_first_n_neurons(energy_model, phase, freq, neurons)
            elif model == "encoder":
                data = generate_data_first_n_neurons(encoder, phase, freq, neurons)
            df = pd.DataFrame({'grating': np.repeat(np.arange(neurons), neurons), 'neuron': np.array(list(np.arange(neurons)) * neurons), 'response': data.flatten()})
            df["phase"] = phase
            df["freq"] = freq
            df["model"] = model
            dataframes.append(df)

first_n_neurons = pd.concat(dataframes, ignore_index=True)

In [6]:
def generate_data_specific_neurons(model, phase, frequency, neurons=[0]):
    """
    Generates gratings and returns the responses in shape (neurons, neurons)
    Beware, responses of all neurons are returned, but only gratings for the first neurons are
    presented (and are stacked into one batch of size neurons, thus the shape (neurons, neurons))
    """
    
    x_size_in_degrees = 11
    x_size = 55
    y_size = 55
    y_size_in_degrees = 11
    inputs = []
    
    for i in neurons:
        inp = imagen.SineGrating(
            orientation = orientations[i],
            frequency = frequency,
            phase = phase,
            bounds = BoundingBox(radius=x_size_in_degrees/2) ,
            offset = 0,
            scale = 1,
            xdensity = x_size/x_size_in_degrees,
            ydensity = y_size/y_size_in_degrees,
        )()
        scaled_array = (inp * (2 + 1.7)) -1.7
        scaled_array = torch.Tensor(scaled_array)
        inputs.append(scaled_array)

    inputs = torch.stack(inputs)
    responses = model(inputs.reshape(len(neurons),1,55,55).to(energy_model.device))
    responses = responses[:, neurons]
    return responses.cpu().detach().numpy()

In [7]:
# find middle neurons
indices = np.where((np.abs(pos_x) < 0.05) & (np.abs(pos_y) < 0.05))
print(indices)

(array([187, 615]),)


In [8]:
neurons = [100, 187, 615]
num_phases = 10
num_freqs = 30

dataframes = []

for phase in np.linspace(0, 2*np.pi, num_phases):
    for freq in np.linspace(0.3, 1.8, num_freqs):
        for model in ["BM", "EM", "encoder"]:
            data = None
            if model == "BM":
                data = generate_data_specific_neurons(bottleneck_model, phase, freq, neurons)
                print(data.shape)
            elif model == "EM":
                data = generate_data_specific_neurons(energy_model, phase, freq, neurons)
            elif model == "encoder":
                data = generate_data_specific_neurons(encoder, phase, freq, neurons)
            df = pd.DataFrame({'grating': np.repeat(np.array(neurons), len(neurons)), 'neuron': np.array(neurons * len(neurons)), 'response': data.flatten()})
            df["phase"] = phase
            df["freq"] = freq
            df["model"] = model
            dataframes.append(df)

specific_neurons = pd.concat(dataframes, ignore_index=True)

(3, 3)


(3, 3)


(3, 3)


(3, 3)


(3, 3)


...

(3, 3)


(3, 3)


(3, 3)


(3, 3)


(3, 3)


In [9]:
table = pd.concat([first_n_neurons, specific_neurons], ignore_index=True)

In [17]:
# join paths experiments and MEI_experiments2.csv
path_to_save = os.path.join(os.path.join("experiments", "MEI_experiments.csv"))
print(f"saved to {path_to_save}")
table.to_csv(path_to_save, index=False)

Error in callback <function _WandbInit._resume_backend at 0x146a7af6d940> (for pre_run_cell):


Exception: The wandb backend process has shutdown

saved to experiments/MEI_experiments.csv
Error in callback <function _WandbInit._pause_backend at 0x146a7af6d670> (for post_run_cell):


Exception: The wandb backend process has shutdown